<a href="https://colab.research.google.com/github/MinKimIP/IPA-public/blob/master/uspto-tm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtain USPTO trademark data

In [0]:
%%capture
!pip install pyarrow
!pip install pandas --upgrade
!pip install xmltodict
!pip install tqdm

In [0]:
import io
import os
import sys
import requests
from zipfile import ZipFile
import glob
import json
import xmltodict
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize # future versions of pandas may move this to: "from pandas import json_normalise"
from tqdm import tqdm

In [0]:
# XML-read dataframe handlers

def normalise(df):
    for column in df.columns:
        try:
            df = df.reset_index(drop=True)
            df = df.join(json_normalize(df[column]).add_prefix(column)).drop(columns=[column])
        except:
            df = df
    
    df = df.pipe(clean_column_names)
    
    return df


def explode(df):
    for column in df.columns:
        try:
            df = df.explode(column)
            df = df.reset_index(drop=True)
        except:
            df = df
    return df


def select_or_create_columns(df, columns):
    df1 = df.copy()
    before = df.columns
    for column in columns:
        if column not in before:
            df1[column] = np.nan
        else:
            df1[column] = df[column]
    
    df1 = df1[columns]
    
    return df1


def clean_column_names(df):
    df.columns = [x.replace('.', '').replace(':', '').replace('#', '').replace('@', '') for x in df.columns]
    return df


def drop_problematic_columns(df, columns):
    for column in columns:
        if column in df.columns:
            df = df.drop(columns = column)
    return df

In [0]:
# make a list of historical zip files

link_base = 'https://bulkdata.uspto.gov/data/trademark/dailyxml/applications/'
zip_name_base = 'apc18840407-20191231'

zip_files = []

for i in range(1, 66):
  zip_files.append(f"{link_base}{zip_name_base}-{str(i).rjust(2, '0')}.zip")

zip_files_batch = zip_files[0:1]

In [24]:
for zip_file in zip_files_batch:
  zip_name = os.path.basename(zip_file).replace('.zip', '')
  save_name = zip_name + '.parquet'
  print(zip_name)
  r = requests.get(zip_file)
  with ZipFile(io.BytesIO(r.content)) as zf:
    xml_files = [x for x in zf.namelist() if x.endswith('.xml')]
    for xml_file in xml_files:
      with zf.open(xml_file) as xf:
        xml = xf.read()
        xmldict = xmltodict.parse(xml)

apc18840407-20191231-01


In [0]:
root = xmldict['trademark-applications-daily']['application-information']['file-segments']['action-keys']
df = pd.DataFrame.from_dict(root, orient='index').T

In [40]:
df

ImportError: ignored

  action-key                                          case-file
0         TX  [{'serial-number': '60000001', 'registration-n...

In [41]:
from pandas.io.common import is_url

ImportError: ignored

In [42]:
pd.__version__

'0.25.3'